<a href="https://colab.research.google.com/github/ncsu-geoforall-lab/GIS582-assignments/blob/main/4AB%20-%20Interpolation/4B_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 4B: Spatial interpolation and approximation II: splines

**Course:** [GIS 582 - Geospatial Modeling and Analysis](https://ncsu-geoforall-lab.github.io/geospatial-modeling-course/index.html)  
**Institution:** [NC State University, Center for Geospatial Analytics](https://cnr.ncsu.edu/geospatial/)
**Instructors:** Helena Mitasova, Corey White, and team

## Learning Objectives

In this tutorial, you will learn how to:
- Interpolate elevation using regularized splines with tension (RST)
- Evaluate the impact of spline tension and smoothing parameters
- Compute and map interpolation deviations and cross-validation error
- Interpolate precipitation with influence of topography

## Tutorial Outline

* Part 1: Environment Setup
* Part 2: Impact of spline tension
* Part 3: Impact of spline smoothing (deviations)
* Part 4: Predictive error (cross-validation)
* Part 5: Precipitation interpolation with topography

---
## Part 1: Environment Setup

### Install GRASS

**Important:** This setup takes 3-5 minutes. You'll need to run it each time you start a new Colab session.

In [ ]:
!add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt-get install -y grass-core grass-dev

Check that GRASS is installed by asking which version is there.

In [ ]:
!grass --version

Check which Python version is running.

In [ ]:
import sys

v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

### Import GRASS Python packages

We need to locate GRASS Python packages using `grass --config python_path` and add them to `sys.path`.

In [ ]:
import subprocess
import os
from pathlib import Path

# Ask GRASS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

import grass.script as gs
import grass.jupyter as gj

### Download North Carolina Sample Dataset

This dataset includes elevation, land cover, roads, streams, interpolation point samples, and precipitation data.

In [ ]:
!grass --tmp-project XY --exec g.download.project url=https://grass.osgeo.org/sampledata/north_carolina/nc_spm_08_grass7.tar.gz path=/content

### Initialize GRASS Session

In [ ]:
# Start GRASS session
grassdata = "/content"
project = "nc_spm_08_grass7"
mapset = "user1"  # Create a new mapset for our work

# Start GRASS Session
session = gj.init(Path(project, mapset))
print("GRASS session started.")

---
## Part 2: Explore impact of spline tension parameter

We will interpolate elevation from scattered points using RST splines and compare surfaces using aspect maps.

We compare:
- Default tension (40)
- Low tension (10)
- High tension (160)

In [ ]:
!g.region rural_1m res=1 -p

Display the input scattered points (elevation stored in attribute column `value`).

In [ ]:
!v.colors map=elev_lid792_randpts use=attr column=value color=elevation
pts_map = gj.Map(height=500, width=500)
pts_map.d_vect(map="elev_lid792_randpts", size=3, type="point")
pts_map.show()

Interpolate elevation and compute aspect simultaneously with different tension values.

In [ ]:
!v.surf.rst input=elev_lid792_randpts elevation=elev_rstdef_1m aspect=asp_rstdef_1m zcolumn=value npmin=140
!v.surf.rst input=elev_lid792_randpts elevation=elev_rstt10_1m aspect=asp_rstt10_1m zcolumn=value tension=10 npmin=140
!v.surf.rst input=elev_lid792_randpts elevation=elev_rstt160_1m aspect=asp_rstt160_1m zcolumn=value tension=160 npmin=140

Assign a consistent aspect color table for comparison.

In [ ]:
!r.colors asp_rstdef_1m color=aspect
!r.colors asp_rstt10_1m color=aspect
!r.colors asp_rstt160_1m color=aspect

Display aspect maps for the three tension settings.

In [ ]:
asp_def_map = gj.Map(filename="asp_rst_t40.png", height=450, width=450)
asp_def_map.d_rast(map="asp_rstdef_1m")
asp_def_map.show()

asp_t10_map = gj.Map(filename="asp_rst_t10.png", height=450, width=450)
asp_t10_map.d_rast(map="asp_rstt10_1m")
asp_t10_map.show()

asp_t160_map = gj.Map(filename="asp_rst_t160.png", height=450, width=450)
asp_t160_map.d_rast(map="asp_rstt160_1m")
asp_t160_map.show()

#### Question 2.1

Explain the impact of the tension parameter by comparing results for tension=40 (default), tension=10, and tension=160.

`Add text here with your answer.`

---

## Part 3: Explore impact of spline smoothing

Explore the impact of the spline smoothing parameter on deviations of the interpolated surface. Compute elevation, aspect, and deviations for two smoothing values (0.1 and 10). Then interpolate the deviations into rasters and compare the spatial patterns.

Create the color-rule files used in this tutorial (equivalent to downloading the provided text files).

In [ ]:
%%bash
cat > deviations_color.txt <<'EOF'
-0.85 red
-0.4 orange
-0.1 yellow
-0.001 220 220 220
0.001 220 220 220
0.1 cyan
0.4 aqua
0.8 blue
1.9 violet
EOF

cat > precip_color.txt <<'EOF'
950 red
1000 orange
1200 yellow
1400 cyan
1600 aqua
1800 blue
2500 violet
EOF

ls -lh deviations_color.txt precip_color.txt

Interpolate with different smoothing parameters and compute deviations point maps.

In [ ]:
!v.surf.rst input=elev_lid792_randpts elevation=elev_rstdef_1mb aspect=asp_rstdef zcolumn=value smooth=0.1 deviations=elev_rstdef_devi npmin=140
!v.surf.rst input=elev_lid792_randpts elevation=elev_rstsm10_1mb aspect=asp_rstsm10 zcolumn=value smooth=10 deviations=elev_rstsm10_devi npmin=140

Compare summary statistics for the deviations (including RMS deviation [`rmsdevi`] from `r.info`).

In [ ]:
!r.info elev_rstdef_1mb
!v.info -c elev_rstdef_devi
!v.univar elev_rstdef_devi column=flt1 type=point

!r.info elev_rstsm10_1mb
!v.info -c elev_rstsm10_devi
!v.univar elev_rstsm10_devi column=flt1 type=point

Interpolate the deviations point maps into raster maps.

In [ ]:
!v.surf.rst input=elev_rstdef_devi elevation=elev_rstdef_devi zcolumn=flt1 npmin=140
!v.surf.rst input=elev_rstsm10_devi elevation=elev_rstsm10_devi zcolumn=flt1 npmin=140

Apply the same divergent custom color rules to both deviation rasters and display them.

In [ ]:
!r.colors elev_rstsm10_devi rules=deviations_color.txt
!r.colors elev_rstdef_devi rules=deviations_color.txt

dev_def_map = gj.Map(filename="elev_rstdef_devi.png", height=450, width=450)
dev_def_map.d_rast(map="elev_rstdef_devi")
dev_def_map.d_legend(raster="elev_rstdef_devi", at=[90, 50, 5, 8])
dev_def_map.show()

dev_sm10_map = gj.Map(filename="elev_rstsm10_devi.png", height=450, width=450)
dev_sm10_map.d_rast(map="elev_rstsm10_devi")
dev_sm10_map.d_legend(raster="elev_rstsm10_devi", at=[90, 50, 5, 8])
dev_sm10_map.show()

#### Question 3.1

What is the difference in deviations (summary statistics and spatial pattern) between smoothing 0.1 and smoothing 10?

`Add text here with your answer.`

---

## Part 4: Compute predictive error of interpolation

Compute predictive error at each point using cross-validation. Then map the predictive error by interpolating it to a raster.

In [ ]:
!v.surf.rst -c input=elev_lid792_randpts zcolumn=value cvdev=elev_rstdef_cv npmin=140
!v.colors elev_rstdef_cv use=attr column=flt1 rules=deviations_color.txt
!v.univar elev_rstdef_cv column=flt1 type=point

Interpolate the cross-validation error to a raster map and display it.

Identify locations where sampling is inadequate (large predictive errors).

In [ ]:
!v.surf.rst input=elev_rstdef_cv elevation=elev_rstdef_cv zcolumn=flt1
!r.colors elev_rstdef_cv rules=deviations_color.txt

cv_map = gj.Map(filename="elev_rstdef_cv.png", height=450, width=450)
cv_map.d_rast(map="elev_rstdef_cv")
cv_map.d_legend(raster="elev_rstdef_cv", at=[90, 50, 5, 8])
cv_map.show()

#### Question 4.1

Where are the largest predictive errors located, and what does that suggest about the sampling pattern?

`Add text here with your answer.`

---
## Part 5: Interpolate precipitation with influence of topography

We will compute precipitation:
- Without elevation influence (2D interpolation)
- With elevation influence (3D spline interpolation + cross-section using a DEM)

We will use a mask to limit computation to North Carolina.

In [ ]:
!g.region raster=elev_state_500m -p
!g.region t=2000 b=0 tbres=2000 res3=500 -p3

Apply the North Carolina mask and compute precipitation without elevation influence.

In [ ]:
!r.mask -r
!r.mask raster=ncmask_500m
!v.info -c precip_30ynormals
!v.surf.rst input=precip_30ynormals elevation=precip_annual_500m zcolumn=annual segmax=700

Apply the custom precipitation color table and display the result.

In [ ]:
!r.colors precip_annual_500m rules=precip_color.txt
precip_map = gj.Map(filename="precip_annual.png", height=450, width=450)
precip_map.d_rast(map="precip_annual_500m")
precip_map.d_legend(raster="precip_annual_500m", at=[90, 60, 5, 8], range=[970, 2400])
precip_map.show()

Compute precipitation with influence of elevation using trivariate splines (`v.vol.rst`) and a cross-section (`cross_output`).

In [ ]:
!v.info -c precip_30ynormals_3d
!v.vol.rst input=precip_30ynormals_3d cross_input=elev_state_500m cross_output=precip_anntopo_500m maskmap=elev_state_500m wcolumn=annual zscale=90 segmax=700
!r.colors precip_anntopo_500m raster=precip_annual_500m

precip_topo_map = gj.Map(filename="precip_anntopo.png", height=450, width=450)
precip_topo_map.d_rast(map="precip_anntopo_500m")
precip_topo_map.d_legend(raster="precip_anntopo_500m", at=[90, 60, 5, 8], range=[970, 2400])
precip_topo_map.show()

Remove the mask when you are finished.

In [ ]:
!r.mask -r

#### Question 5.1

Try to explain how elevation was used for the precipitation interpolation.

`Add text here with your answer.`